In [23]:
# Dependencies and setup
import pandas as pd
import numpy as np
import requests
from pathlib import Path
import matplotlib.pyplot as plt

from api_keys import census_key

In [26]:
# Import unemployment rate csv file, create DataFrame, and set index to Area Type
unemployment_data = Path('resources/Local_Area_Unemployment_Statistics__LAUS___Annual_Average.csv')
unemployment_df = pd.read_csv(unemployment_data)
unemployment_df = unemployment_df.set_index('Area Type')
unemployment_df

# Filter Area Type to only include County and State data and reduce columns
area_type_df = unemployment_df.loc[['State','County'],
                                   ['Area Name', 'Year', 'Labor Force', 'Employment', 'Unemployment','Unemployment Rate']]
area_type_df

# Group by Year and calculate averages
year_grouped = area_type_df.groupby(['Year', 'Area Name']).mean()
year_grouped

# Reduce to only 2017-2022
recent = year_grouped.loc[[2017, 2018, 2019, 2020, 2021, 2022]]
recent

Labor Force  Employment  Unemployment  \
Year Area Name                                                 
2017 Alameda County       838200.0    807100.0       31100.0   
     Alpine County           550.0       510.0          30.0   
     Amador County         14690.0     13960.0         740.0   
     Butte County         102000.0     96100.0        5900.0   
     Calaveras County      21020.0     20010.0        1000.0   
...                            ...         ...           ...   
2022 Tulare County        207500.0    190500.0       16900.0   
     Tuolumne County       19880.0     18990.0         890.0   
     Ventura County       413600.0    398400.0       15200.0   
     Yolo County          109000.0    104600.0        4400.0   
     Yuba County           31600.0     29900.0        1700.0   

                       Unemployment Rate  
Year Area Name                            
2017 Alameda County                  3.7  
     Alpine County                   6.2  
     Amador County                   5.0  
     Butte County                    5.8  
     Calaveras County                4.8  
...                                  ...  
2022 Tulare County                   8.2  
     Tuolumne County                 4.5  
     Ventura County                  3.7  
     Yolo County                     4.0  
     Yuba County                     5.5  

[354 rows x 4 columns]

In [3]:
# Import age csv file and create DataFrame
age_data = Path('resources/experiencing_homelessness_age_demographics.csv')
age_df = pd.read_csv(age_data)
age_df

,CALENDAR_YEAR,COC_ID,COC_NAME,AGE_GROUP_PUBLIC,EXPERIENCING_HOMELESSNESS_CNT
0,2017,All,California,18-24,15862
1,2017,All,California,25-34,28339
2,2017,All,California,35-44,25556
3,2017,All,California,45-54,27441
4,2017,All,California,55-64,23220
...,...,...,...,...,...
2136,2022,CA-614,San Luis Obispo County CoC,45-54,389
2137,2022,CA-614,San Luis Obispo County CoC,55-64,360
2138,2022,CA-614,San Luis Obispo County CoC,65+,174
2139,2022,CA-614,San Luis Obispo County CoC,Under 18,742


In [4]:
# Create clean DataFrame by dropping rows with '*' in count column
clean_age_df = age_df.loc[age_df['EXPERIENCING_HOMELESSNESS_CNT']!='*']
clean_age_df

,CALENDAR_YEAR,COC_ID,COC_NAME,AGE_GROUP_PUBLIC,EXPERIENCING_HOMELESSNESS_CNT
0,2017,All,California,18-24,15862
1,2017,All,California,25-34,28339
2,2017,All,California,35-44,25556
3,2017,All,California,45-54,27441
4,2017,All,California,55-64,23220
...,...,...,...,...,...
2136,2022,CA-614,San Luis Obispo County CoC,45-54,389
2137,2022,CA-614,San Luis Obispo County CoC,55-64,360
2138,2022,CA-614,San Luis Obispo County CoC,65+,174
2139,2022,CA-614,San Luis Obispo County CoC,Under 18,742


In [5]:
# Change homeless count data type to integer
clean_age_df['EXPERIENCING_HOMELESSNESS_CNT'] = clean_age_df["EXPERIENCING_HOMELESSNESS_CNT"].astype(int)

# Group by year and COC Name and sum homeless counts
total_homeless_count = clean_age_df.groupby(['CALENDAR_YEAR', 'COC_NAME'])['EXPERIENCING_HOMELESSNESS_CNT'].sum().reset_index()
total_homeless_count

C:\Users\mattcalvert\AppData\Local\Temp\ipykernel_11940\2318218614.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  clean_age_df['EXPERIENCING_HOMELESSNESS_CNT'] = clean_age_df["EXPERIENCING_HOMELESSNESS_CNT"].astype(int)


,CALENDAR_YEAR,COC_NAME,EXPERIENCING_HOMELESSNESS_CNT
0,2017,Alameda County CoC,6713
1,2017,"Alpine, Inyo, Mono Counties CoC",151
2,2017,"Amador, Calaveras, Mariposa, Tuolumne Counties...",368
3,2017,Butte County CoC,1400
4,2017,California,184234
...,...,...,...
263,2022,Stanislaus County CoC,6482
264,2022,Tehama County CoC,670
265,2022,Ventura County CoC,2919
266,2022,Yolo County CoC,1419


In [6]:
# Find all Continuums of Care in Los Angeles County
la_county_coc = total_homeless_count.loc[total_homeless_count["COC_NAME"].str.contains("Los Angeles County", case = False)]
la_county_coc

,CALENDAR_YEAR,COC_NAME,EXPERIENCING_HOMELESSNESS_CNT
9,2017,Glendale CoC (Los Angeles County),341
15,2017,Long Beach CoC (Los Angeles County),2754
16,2017,Los Angeles County CoC,57002
24,2017,Pasadena CoC (Los Angeles County),1328
53,2018,Glendale CoC (Los Angeles County),362
59,2018,Long Beach CoC (Los Angeles County),2746
60,2018,Los Angeles County CoC,76037
68,2018,Pasadena CoC (Los Angeles County),1352
97,2019,Glendale CoC (Los Angeles County),431
103,2019,Long Beach CoC (Los Angeles County),3265


In [7]:
# Use replace to group all of the Los Angeles County CoCs
cocs_to_replace = ["Glendale CoC (Los Angeles County)",
                   "Pasadena CoC (Los Angeles County)", 
                   "Long Beach CoC (Los Angeles County)"
                  ]
total_homeless_count['COC_NAME'] = total_homeless_count['COC_NAME'].replace(cocs_to_replace, "Los Angeles County CoC")
total_homeless_count

,CALENDAR_YEAR,COC_NAME,EXPERIENCING_HOMELESSNESS_CNT
0,2017,Alameda County CoC,6713
1,2017,"Alpine, Inyo, Mono Counties CoC",151
2,2017,"Amador, Calaveras, Mariposa, Tuolumne Counties...",368
3,2017,Butte County CoC,1400
4,2017,California,184234
...,...,...,...
263,2022,Stanislaus County CoC,6482
264,2022,Tehama County CoC,670
265,2022,Ventura County CoC,2919
266,2022,Yolo County CoC,1419


In [8]:
# Use Groupby again to group new Los Angeles County CoC rows
total_homeless_count = total_homeless_count.groupby(['CALENDAR_YEAR','COC_NAME'])['EXPERIENCING_HOMELESSNESS_CNT'].sum().reset_index()
total_homeless_count

,CALENDAR_YEAR,COC_NAME,EXPERIENCING_HOMELESSNESS_CNT
0,2017,Alameda County CoC,6713
1,2017,"Alpine, Inyo, Mono Counties CoC",151
2,2017,"Amador, Calaveras, Mariposa, Tuolumne Counties...",368
3,2017,Butte County CoC,1400
4,2017,California,184234
...,...,...,...
245,2022,Stanislaus County CoC,6482
246,2022,Tehama County CoC,670
247,2022,Ventura County CoC,2919
248,2022,Yolo County CoC,1419


In [9]:
# Import california population csv file and create DataFrame
population_data = Path('resources/california_population.csv')
population_df = pd.read_csv(population_data)
population_df

,COUNTY,Year,Total Population,Total Housing Units,Single Detached,Single Attached,Two to Four,Five Plus,Mobile Homes,Occupied,Vacancy Rate,Persons per Household
0,Alameda,2017,"1,644,303","596,937","315,395","46,333","65,721","161,630","7,858","564,303",5.50%,2.84
1,Alpine,2017,"1,161","1,780","1,052",18,45,631,34,483,72.90%,2.35
2,Amador,2017,"36,900","18,189","14,905",570,605,690,"1,419","15,140",16.80%,2.18
3,Butte,2017,"225,468","98,871","61,689","3,112","8,976","10,855","14,239","90,791",8.20%,2.42
4,Calaveras,2017,"45,170","27,908","24,256",366,676,392,"2,218","18,371",34.20%,2.43
...,...,...,...,...,...,...,...,...,...,...,...,...
349,Tuolumne,2022,"54,465","31,522","25,225",311,"1,211","1,437","3,338","23,014",27.00%,2.23
350,Ventura,2022,"831,533","294,989","187,223","33,143","16,048","47,700","10,875","281,120",4.70%,2.91
351,Yolo,2022,"221,639","81,945","49,312","5,107","6,572","17,390","3,564","78,135",4.60%,2.62
352,Yuba,2022,"82,193","30,214","21,764",570,"2,640","2,356","2,884","27,775",8.10%,2.88


In [10]:
# Reduce columns to just county, year, and total population
population_df = population_df[['COUNTY','Year','Total Population']]
population_df

,COUNTY,Year,Total Population
0,Alameda,2017,"1,644,303"
1,Alpine,2017,"1,161"
2,Amador,2017,"36,900"
3,Butte,2017,"225,468"
4,Calaveras,2017,"45,170"
...,...,...,...
349,Tuolumne,2022,"54,465"
350,Ventura,2022,"831,533"
351,Yolo,2022,"221,639"
352,Yuba,2022,"82,193"


In [11]:
# Rename COUNTY column
population_df = population_df.rename(columns = {'COUNTY': 'County'})
population_df

,County,Year,Total Population
0,Alameda,2017,"1,644,303"
1,Alpine,2017,"1,161"
2,Amador,2017,"36,900"
3,Butte,2017,"225,468"
4,Calaveras,2017,"45,170"
...,...,...,...
349,Tuolumne,2022,"54,465"
350,Ventura,2022,"831,533"
351,Yolo,2022,"221,639"
352,Yuba,2022,"82,193"


In [29]:
#add 'County' to the county names
population_df['County'] = population_df['County'].astpye(str) + str(' County')
population_df

KeyError: 'County'